In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np

from pathlib import Path
import os
import zipfile
import matplotlib.pyplot as plt

# Read data

In [3]:
muni = pd.read_csv('/home/juliane.oliveira/Documents/AESOP data structure/ETL_data/DTB_BRASIL_MUNICIPIO.csv',sep = ';')
hubs = pd.read_csv('/home/juliane.oliveira/workspace/Reproducibility-results_paper2/data/lists_of_hubs.csv')

link0 = '/home/juliane.oliveira/Documents/GitHub/aesop-models/scripts/Mobility pipeline/adjacency_matrix_correct.parquet'
matrix = pd.read_parquet(link0, engine='pyarrow')

# Covid-19 cases from Wesley Cota
covid = pd.read_csv('/home/juliane.oliveira/workspace/Reproducibility-results_paper2/data/municipios.csv')

path = pd.read_csv('/home/juliane.oliveira/workspace/Reproducibility-results_paper2/data/union_paths.csv')

In [4]:
path

,Unnamed: 0.1,Unnamed: 0,value_cum,value,ori_muni_name,ori_uf_name,ori_co_ibge,des_muni_name,des_uf_name,des_co_ibge,path_correct,path_len,muni_1,muni_2,muni_3,ones
0,0,0,56.000000,56.000000,Acrelândia,Acre,1200013,Cruzeiro do Sul,Acre,1200203,"[52, 66, 57]",2,66,57.0,NaN,1
1,1,1,28.000000,28.000000,Assis Brasil,Acre,1200054,Rodrigues Alves,Acre,1200427,"[53, 66, 57, 67]",3,66,57.0,67.0,1
2,2,2,168.000000,77.000000,Brasiléia,Acre,1200104,Sena Madureira,Acre,1200500,"[54, 56, 66, 70]",3,56,66.0,70.0,1
3,3,3,15.108333,8.000000,Bujari,Acre,1200138,Lábrea,Amazonas,1302405,"[55, 66, 1, 99, 108]",4,66,1.0,99.0,1
4,4,4,47.925000,33.004167,Capixaba,Acre,1200179,Parintins,Amazonas,1303403,"[56, 66, 16, 111, 119]",4,66,16.0,111.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6734812,877165,877165,836.566667,53.566667,São Paulo,São Paulo,3550308,Pedra Branca,Ceará,2310506,"[3829, 949, 1055, 1027]",3,949,1055.0,1027.0,1
6734813,877166,877166,561.954167,172.000000,São Paulo,São Paulo,3550308,Araripina,Pernambuco,2601102,"[3829, 1590, 1477]",2,1590,1477.0,NaN,1
6734814,877167,877167,527.566667,147.000000,São Paulo,São Paulo,3550308,Itagimirim,Bahia,2915304,"[3829, 2137, 2008]",2,2137,2008.0,NaN,1
6734815,877168,877168,398.000000,149.000000,São Paulo,São Paulo,3550308,Inhapim,Minas Gerais,3130903,"[3829, 2558, 2595]",2,2558,2595.0,NaN,1


# Data preparation

In [5]:
covid = covid[['UF', 'city', 'date', 'ibgeID', 'newCases', 'mortes', 'TOTAL']]
covid['ibgeID'] = covid['ibgeID'].astype(int)
covid['UF'] = covid.city.astype(str).str[-2:]

In [6]:
muni = muni.assign(co_ibge_6 = muni['Código Município Completo'].astype(str).str[:6])
muni = muni[['UF', 'Nome_UF', 'Mesorregião Geográfica', 'Nome_Mesorregião',
       'Microrregião Geográfica', 'Nome_Microrregião', 'Município',
       'Código Município Completo', 'Nome_Município','co_ibge_6']]

# Functions

In [7]:
link_muni_vertice = pd.DataFrame(matrix.columns, columns=['muni'])

In [8]:
def get_mname(n):
    
    m = link_muni_vertice.iloc[n]['muni']
    set_muni = muni[muni['Código Município Completo'] == m].reset_index()
    return [set_muni.iloc[0]['Nome_Município'],set_muni.iloc[0]['Nome_UF'],m]
   
def get_mnumber(name):
    muni[muni['Nome_Município'] == name]
    
    co_mu = muni[muni['Nome_Município'] == name].reset_index()['Código Município Completo'][0]
    muni_number = link_muni_vertice[link_muni_vertice['muni'] == co_mu]['muni'].index.tolist()[0]
    return [muni_number, co_mu]

In [9]:
df_np = matrix.to_numpy()

In [10]:
def col_name(dtf,col):
    lst = []
    for value in col:
        muni_name = get_mname(value)[0]
        uf_muni = get_mname(value)[1]
        cod_ibge_muni = get_mname(value)[2]
    
        lst.append([muni_name,uf_muni,cod_ibge_muni])
    
    dta = pd.DataFrame(lst, columns=['muni_name','uf_muni','cod_ibge_muni'])
    
    dtf = dtf.assign(muni_name = dta.muni_name)
    dtf = dtf.assign(uf_muni = dta.uf_muni)
    dtf = dtf.assign(cod_ibge_muni = dta.cod_ibge_muni)
    
    return dtf

# Correlation of spread from one selected city

In [11]:
covid.date.min()

'2020-02-25'

In [16]:
fst_inv.groupby(['city'])['newCases'].sum().reset_index().sort_values(by='newCases')

,city,newCases
0,Barra Mansa/RJ,1.0
17,Rio Verde/GO,1.0
2,Campo Bom/RS,1.0
3,Caxias do Sul/RS,1.0
4,Cianorte/PR,1.0
6,Divinópolis/MG,1.0
8,Ferraz de Vasconcelos/SP,1.0
14,Niterói/RJ,1.0
19,Santana de Parnaíba/SP,1.0
11,Ipatinga/MG,1.0


In [17]:
fst_inv = covid[covid.date <= '2020-03-12']
dta_fst = fst_inv.groupby(['city','ibgeID'])['newCases'].sum().reset_index().sort_values(by= ['newCases'])

sec_inv = covid[(covid.date < '2020-04-01')]#covid[(covid.date > '2020-03-12') & (covid.date < '2020-04-01')]
dta_sec = sec_inv.groupby(['city','ibgeID'])['newCases'].sum().reset_index().sort_values(by= ['newCases'])

th_inv = covid[(covid.date < '2020-04-16')]
dta_th = th_inv.groupby(['city','ibgeID'])['newCases'].sum().reset_index().sort_values(by= ['newCases'])

ft_inv = covid[(covid.date < '2020-05-01')]
dta_ft = ft_inv.groupby(['city','ibgeID'])['newCases'].sum().reset_index().sort_values(by= ['newCases'])

# Analysis of three step path infection

In [48]:
city = 'São Paulo'
uf = 'São Paulo'

path_city = path[(path.ori_muni_name == city)& (path.ori_uf_name == uf)]

In [49]:
dta1 = path_city.groupby(['muni_1'])['ones'].sum().reset_index()
dta2 = path_city.groupby(['muni_2'])['ones'].sum().reset_index()
dta3 = path_city.groupby(['muni_3'])['ones'].sum().reset_index()

In [50]:
dta1 = col_name(dta1,dta1.muni_1)
dta1 = dta1.rename(columns={"muni_name": "muni1_name",'uf_muni':'uf_muni1','cod_ibge_muni':'cod_ibge_muni1'})

dta2.muni_2 = dta2.muni_2.astype(int)
dta2 = col_name(dta2,dta2.muni_2)
dta2 = dta2.rename(columns={"muni_name": "muni2_name",'uf_muni':'uf_muni2','cod_ibge_muni':'cod_ibge_muni2'})

dta3.muni_3 = dta3.muni_3.astype(int)
dta3 = col_name(dta3,dta3.muni_3)
dta3 = dta3.rename(columns={"muni_name": "muni3_name",'uf_muni':'uf_muni3','cod_ibge_muni':'cod_ibge_muni3'})

In [51]:
# Muni that are in second step and are not in the first

st1 = dta1.cod_ibge_muni1
st2 = dta2.cod_ibge_muni2
st3 = dta3.cod_ibge_muni3

lst1 = list(st1) + [3550308]#cod Rio[3304557], SP [3550308]
lst2_1 = list(set(st2) - set(st1))
lst3_21 = set(st3) - (set(st2) | set(st1))

In [52]:
# Find the infected cities in step 1, 2 and 3 in the first period
infect_muni1_P1 = sec_inv[sec_inv.ibgeID.isin(lst1)]
infect_muni2_P1 = sec_inv[sec_inv.ibgeID.isin(lst2_1)]
infect_muni3_P1 = sec_inv[sec_inv.ibgeID.isin(lst3_21)]

In [53]:
# Find the proportion of cases in the cities in step 1, 2 and 3 compared to the total number of casesin the state

In [54]:
dta11 = infect_muni1_P1.groupby(['UF'])['newCases'].sum().reset_index()
dta11 = dta11.rename(columns={'newCases':'CasesInUfMuni1_P1'})

dta21 = infect_muni2_P1.groupby(['UF'])['newCases'].sum().reset_index()
dta21 = dta21.rename(columns={'newCases':'CasesInUfMuni2_P1'})

dta31 = infect_muni3_P1.groupby(['UF'])['newCases'].sum().reset_index()
dta31 = dta31.rename(columns={'newCases':'CasesInUfMuni3_P1'})

In [55]:
dta=sec_inv.groupby(['UF'])['newCases'].sum().reset_index().sort_values(by = 'newCases')

In [56]:
result11 = dta.set_index('UF').join(dta11.set_index('UF')).reset_index()
result21 = result11.set_index('UF').join(dta21.set_index('UF')).reset_index()
result31 = result21.set_index('UF').join(dta31.set_index('UF')).reset_index()

In [57]:
result31 = result31.fillna(0)
result31.head()

,UF,newCases,CasesInUfMuni1_P1,CasesInUfMuni2_P1,CasesInUfMuni3_P1
0,RO,9.0,0.0,8.0,0.0
1,AP,10.0,0.0,0.0,0.0
2,TO,12.0,9.0,0.0,0.0
3,AL,13.0,13.0,0.0,0.0
4,RR,13.0,0.0,0.0,0.0


In [58]:
result31 = result31.assign(prop_muni1_P1 = round(result31.CasesInUfMuni1_P1*100/result31.newCases))
result31 = result31.assign(prop_muni2_P1 = round(result31.CasesInUfMuni2_P1*100/result31.newCases))
result31 = result31.assign(prop_muni3_P1 = round(result31.CasesInUfMuni3_P1*100/result31.newCases))

In [59]:
result31.describe()

,newCases,CasesInUfMuni1_P1,CasesInUfMuni2_P1,CasesInUfMuni3_P1,prop_muni1_P1,prop_muni2_P1,prop_muni3_P1
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,206.037037,167.333333,11.592593,0.555556,64.222222,15.740741,1.148148
std,443.604855,382.623836,19.619965,1.476309,36.677737,29.784602,4.148250
min,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,18.000000,11.000000,0.000000,0.000000,42.500000,0.000000,0.000000
50%,62.000000,39.000000,4.000000,0.000000,78.000000,3.000000,0.000000
75%,199.000000,163.500000,12.500000,0.000000,92.000000,11.000000,0.000000
max,2279.000000,1949.000000,76.000000,7.000000,100.000000,93.000000,21.000000


In [63]:
result31[result31.prop_muni1_P1 >= result31.prop_muni2_P1]#.describe()
#result31[result31.prop_muni1_sec_inv < result31.prop_muni2_sec_inv]

,UF,newCases,CasesInUfMuni1_P1,CasesInUfMuni2_P1,CasesInUfMuni3_P1,prop_muni1_P1,prop_muni2_P1,prop_muni3_P1
1,AP,10.0,0.0,0.0,0.0,0.0,0.0,0.0
2,TO,12.0,9.0,0.0,0.0,75.0,0.0,0.0
3,AL,13.0,13.0,0.0,0.0,100.0,0.0,0.0
4,RR,13.0,0.0,0.0,0.0,0.0,0.0,0.0
5,PI,17.0,16.0,0.0,0.0,94.0,0.0,0.0
7,SE,19.0,18.0,1.0,0.0,95.0,5.0,0.0
8,MT,24.0,20.0,4.0,0.0,83.0,17.0,0.0
9,PA,34.0,21.0,3.0,7.0,62.0,9.0,21.0
10,MS,39.0,39.0,0.0,0.0,100.0,0.0,0.0
12,MA,51.0,51.0,0.0,0.0,100.0,0.0,0.0


In [64]:
result31[result31.prop_muni1_P1 < result31.prop_muni2_P1].describe()

,newCases,CasesInUfMuni1_P1,CasesInUfMuni2_P1,CasesInUfMuni3_P1,prop_muni1_P1,prop_muni2_P1,prop_muni3_P1
count,4.000000,4.000000,4.000000,4.00000,4.000000,4.000000,4.000000
mean,35.750000,1.000000,31.250000,0.50000,4.750000,84.750000,1.750000
std,29.702693,1.414214,28.182441,0.57735,8.845903,7.675719,2.872281
min,9.000000,0.000000,8.000000,0.00000,0.000000,76.000000,0.000000
25%,15.000000,0.000000,11.750000,0.00000,0.000000,79.750000,0.000000
50%,29.500000,0.500000,23.500000,0.50000,0.500000,85.000000,0.500000
75%,50.250000,1.500000,43.000000,1.00000,5.250000,90.000000,2.250000
max,75.000000,3.000000,70.000000,1.00000,18.000000,93.000000,6.000000


In [ ]:
RO, PB, AC, RN

In [252]:
result31.to_csv('/home/juliane.oliveira/workspace/Reproducibility-results_paper2/data/SP_path_asso_P1.csv')

In [193]:
# Find the infected cities in step 1, 2 and 3 in the second period
infect_muni1_P2 = th_inv[th_inv.ibgeID.isin(lst1)]
infect_muni2_P2 = th_inv[th_inv.ibgeID.isin(lst2_1)]
infect_muni3_P2 = th_inv[th_inv.ibgeID.isin(lst3_21)]

In [194]:
# Find the proportion of cases in the cities in step 1, 2 and 3 compared to the total number of cases in the state

In [195]:
dta12 = infect_muni1_P2.groupby(['UF'])['newCases'].sum().reset_index()
dta12 = dta12.rename(columns={'newCases':'CasesInUfMuni1_P2'})

dta22 = infect_muni2_P2.groupby(['UF'])['newCases'].sum().reset_index()
dta22 = dta22.rename(columns={'newCases':'CasesInUfMuni2_P2'})

dta32 = infect_muni3_P2.groupby(['UF'])['newCases'].sum().reset_index()
dta32 = dta32.rename(columns={'newCases':'CasesInUfMuni3_P2'})

In [196]:
dta = th_inv.groupby(['UF'])['newCases'].sum().reset_index().sort_values(by = 'newCases')

In [197]:
result12 = dta.set_index('UF').join(dta12.set_index('UF')).reset_index()
result22 = result12.set_index('UF').join(dta22.set_index('UF')).reset_index()
result32 = result22.set_index('UF').join(dta32.set_index('UF')).reset_index()

In [198]:
result32 = result32.fillna(0)
result32.head()

,UF,newCases,CasesInUfMuni1_P2,CasesInUfMuni2_P2,CasesInUfMuni3_P2
0,TO,27.0,0.0,26.0,1.0
1,SE,46.0,0.0,39.0,4.0
2,RO,73.0,0.0,59.0,14.0
3,AL,75.0,67.0,2.0,1.0
4,PI,82.0,78.0,2.0,2.0


In [199]:
result32 = result32.assign(prop_muni1_P2 = round(result32.CasesInUfMuni1_P2*100/result32.newCases))
result32 = result32.assign(prop_muni2_P2 = round(result32.CasesInUfMuni2_P2*100/result32.newCases))
result32 = result32.assign(prop_muni3_P2 = round(result32.CasesInUfMuni3_P2*100/result32.newCases))

In [206]:
result31[result31.prop_muni1_P1 < result31.prop_muni2_P1]

,UF,newCases,CasesInUfMuni1_P1,CasesInUfMuni2_P1,CasesInUfMuni3_P1,prop_muni1_P1,prop_muni2_P1,prop_muni3_P1
0,RO,9.0,0.0,7.0,2.0,0.0,78.0,22.0
1,AP,10.0,0.0,10.0,0.0,0.0,100.0,0.0
2,TO,12.0,0.0,12.0,0.0,0.0,100.0,0.0
4,RR,13.0,0.0,13.0,0.0,0.0,100.0,0.0
7,SE,19.0,0.0,18.0,1.0,0.0,95.0,5.0
8,MT,24.0,4.0,20.0,0.0,17.0,83.0,0.0
11,AC,42.0,0.0,34.0,0.0,0.0,81.0,0.0
14,RN,75.0,0.0,70.0,3.0,0.0,93.0,4.0
18,AM,174.0,0.0,159.0,7.0,0.0,91.0,4.0
19,SC,192.0,58.0,86.0,28.0,30.0,45.0,15.0


In [204]:
#result32[result32.prop_muni1_P2 >= result32.prop_muni2_P2]
result32[result32.prop_muni1_P2 < result32.prop_muni2_P2]

,UF,newCases,CasesInUfMuni1_P2,CasesInUfMuni2_P2,CasesInUfMuni3_P2,prop_muni1_P2,prop_muni2_P2,prop_muni3_P2
0,TO,27.0,0.0,26.0,1.0,0.0,96.0,4.0
1,SE,46.0,0.0,39.0,4.0,0.0,85.0,9.0
2,RO,73.0,0.0,59.0,14.0,0.0,81.0,19.0
6,AC,101.0,0.0,76.0,2.0,0.0,75.0,2.0
7,MT,135.0,23.0,92.0,20.0,17.0,68.0,15.0
8,RR,139.0,0.0,131.0,0.0,0.0,94.0,0.0
11,AP,340.0,0.0,290.0,0.0,0.0,85.0,0.0
12,RN,367.0,0.0,312.0,13.0,0.0,85.0,4.0
18,SC,709.0,214.0,281.0,115.0,30.0,40.0,16.0
23,AM,1555.0,0.0,1350.0,26.0,0.0,87.0,2.0


In [205]:
result32.to_csv('/home/juliane.oliveira/workspace/Reproducibility-results_paper2/data/RJ_path_asso_P2.csv')

Como ainda não temos os resultados de SP, vou ver como está se comportando o espalhamento considerando os paths do rio

## Get infected cities in path of transmission of selected city

In [42]:
x_1 = dta_sec.ibgeID.to_numpy() # cities in the second period of covid spread

x_2  = dta_th.ibgeID.to_numpy() # cities in the third period of covid spread

x_3 = dta_ft.ibgeID.to_numpy()
    
print(len(x_1), len(x_2), len(x_3))

280 598 891


In [43]:
def get_infec_propa_path(df):
    
    p1_inter = []
    for value in df.cod_ibge_muni:
        if value in x_1: x = 1
        else: x = 0
    
        p1_inter.append(x)
    
    p2_inter = []
    for value in df.cod_ibge_muni:
        if value in x_2: x = 1
        else: x = 0
    
        p2_inter.append(x)
    
    p3_inter = []
    for value in df.cod_ibge_muni:
        if value in x_3: x = 1
        else: x = 0
    
        p3_inter.append(x)

    df = df.assign(infect_p1 = p1_inter)
    df = df.assign(infect_p2 = p2_inter)
    df = df.assign(infect_p3 = p3_inter)

    df = df.assign(new_infect_p2 = df.infect_p2 - df.infect_p1)
    df = df.assign(new_infect_p3 = df.infect_p3 - (df.infect_p1 + df.new_infect_p2))
    
    return df

In [44]:
def inf_path_per(city):

    path_uf = path[path.ori_muni_name == city]

    dta1 = path_uf.groupby(['muni_1'])['ones'].sum().reset_index()
    dta2 = path_uf.groupby(['muni_2'])['ones'].sum().reset_index()
    dta3 = path_uf.groupby(['muni_3'])['ones'].sum().reset_index()

    dta1 = col_name(dta1,dta1.muni_1)
    dta1 = dta1.assign(per = round(dta1.ones*100/sum(dta1.ones),1))
    dta1 = dta1.assign(ori_muni = city)

    dta2.muni_2 = dta2.muni_2.astype(int)
    dta2 = col_name(dta2,dta2.muni_2)
    dta2 = dta2.assign(per = round(dta2.ones*100/sum(dta2.ones),1))
    dta2 = dta2.assign(ori_muni = city)

    dta3.muni_3 = dta3.muni_3.astype(int)
    dta3 = col_name(dta3,dta3.muni_3)
    dta3 = dta3.assign(per = round(dta3.ones*100/sum(dta3.ones),1))
    dta3 = dta3.assign(ori_muni = city)

    x1 = dta1.cod_ibge_muni.to_numpy() # fisrt step cities for spread after emergency in selected municipality

    x2 = dta2.cod_ibge_muni.to_numpy() # second step cities for spread after emergency in selected municipality

    x3 = dta3.cod_ibge_muni.to_numpy()
    
    dta1 = get_infec_propa_path(dta1)
    dta2 = get_infec_propa_path(dta2)
    dta3 = get_infec_propa_path(dta3)
    
    n11 = len(np.intersect1d(x1, x_1)) # Number of cities that are the first step for the selected city and are in the second period of spread
    n12 = len(np.intersect1d(x1, x_2))
    n13 = len(np.intersect1d(x1, x_3))

    n21 = len(np.intersect1d(x2, x_1)) 
    n22 = len(np.intersect1d(x2, x_2))
    n23 = len(np.intersect1d(x2, x_3))

    n31 = len(np.intersect1d(x3, x_1)) 
    n32 = len(np.intersect1d(x3, x_2))
    n33 = len(np.intersect1d(x3, x_3))
    
    d = {'path_len': [len(x1), len(x2),len(x3)], 'infect_path1': [n11, n12,n13],
    'infect_path2': [n21, n22,n23], 'infect_path3': [n31, n32,n33]}

    summary = pd.DataFrame(data=d)
    
    #print('x1', len(x1), 'x2', len(x2), 'x3', len(x3))

    #print(round(n11*100/len(x1),1), round(n12*100/len(x1),1), round(n13*100/len(x1),1))
    
    #print(round(n21*100/len(x2),1), round(n22*100/len(x2),1), round(n23*100/len(x2),1))

    #print(round(n31*100/len(x3),1), round(n32*100/len(x3),1), round(n33*100/len(x3),1))
    
    return [summary, dta1,dta2,dta3]

In [46]:
inf_path_per('Rio de Janeiro')[0]

,path_len,infect_path1,infect_path2,infect_path3
0,83,47,135,54
1,892,62,250,118
2,669,66,309,185


In [30]:
df_result = inf_path_per('Rio de Janeiro')[3]

df_result = df_result.rename(columns={"muni_name": 'muni_3_name', 
                                      "uf_muni": "uf_muni_3",
                                      "cod_ibge_muni": "cod_ibge_muni_3"})

df_result.head()

,muni_3,ones,muni_3_name,uf_muni_3,cod_ibge_muni_3,per,ori_muni,infect_p1,infect_p2,infect_p3,new_infect_p2,new_infect_p3
0,1,1,Ariquemes,Rondônia,1100023,0.1,Rio de Janeiro,0,1,1,1,0
1,3,1,Cacoal,Rondônia,1100049,0.1,Rio de Janeiro,0,0,1,0,1
2,4,1,Cerejeiras,Rondônia,1100056,0.1,Rio de Janeiro,0,0,0,0,0
3,5,2,Colorado do Oeste,Rondônia,1100064,0.3,Rio de Janeiro,0,0,0,0,0
4,10,1,Jaru,Rondônia,1100114,0.1,Rio de Janeiro,1,1,1,0,0


In [31]:
df_result.to_csv('/home/juliane.oliveira/workspace/Reproducibility-results_paper2/data/valida_covid_spread_path_step_3_RJ.csv')